Binary Classification의 대표적인 문제
1. 위스콘신 유방암 데이터
2. Titanic

In [20]:
# 위스콘신 유방암 데이터로 구현
# sklearn에서 제공함
# sklearn과 tensorfow를 이용해서 구현

import numpy as np
from sklearn import linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split # tr
from sklearn.model_selection import cross_val_score
import warnings

# 경고메시지 출력하지 않아요!
warnings.filterwarnings(action='ignore')

In [8]:
# Raw data loading
cancer = load_breast_cancer()
# print(type(cancer))  # <class 'sklearn.utils.Bunch'>
                       # sklearn이 데이터를 표현하기 위해 사용하는 자료구조
                       # python의 dictionary와 유사한 구조
#print(cancer)
# data라는 
#print(cancer.data.shape, cancer.target.shape)  #(569, 30) (569,)
#print(np.unique(cancer.target, return_counts=True))
#(array([0, 1]), array([212, 357], dtype=int64))
print(cancer.DESCR)  # 유방암데이터에대한 상세 내용!
# :Missing Attribute Values: None
# :Class Distribution: 212 - Malignant(악성), 357 - Benign(정상)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [11]:
# data set
x_data = cancer.data
t_data = cancer.target

print(type(x_data), x_data.shape)   #<class 'numpy.ndarray'> (569, 30) 

<class 'numpy.ndarray'> (569, 30)


In [16]:
# hold-out validation을 위해서 train과 validation데이터를 분리
train_x_data, test_x_data, train_t_data, test_t_data = train_test_split(x_data,t_data,test_size=0.2, random_state=2,stratify=t_data)
print(train_x_data.shape, train_t_data.shape)  #(455, 30) (455,)
print(np.unique(train_t_data, return_counts=True))   #(array([0, 1]), array([170, 285], dtype=int64))

(455, 30) (455,)
(array([0, 1]), array([170, 285], dtype=int64))


In [17]:
#모델 생성
model = linear_model.LogisticRegression()

In [21]:
# k-fold cross validation
test_score = cross_val_score(model, x_data, t_data, scoring='accuracy', cv=5)
print(test_score)  #[0.92982456 0.93859649 0.96491228 0.95614035 0.95575221]
print(test_score.mean())   #0.9490451793199813

[0.92982456 0.93859649 0.96491228 0.95614035 0.95575221]
0.9490451793199813


In [22]:
# hold-out 방식으로 validation
model.fit(train_x_data,train_t_data)
test_score = model.score(test_x_data, test_t_data)
print(test_score)   #0.956140350877193

0.956140350877193


In [23]:
import tensorflow as tf

In [25]:
# 이제 tensorflow로 돌려보기!
## placeholder
X = tf.placeholder(shape=[None,30],dtype=tf.float32)
T = tf.placeholder(shape=[None,1],dtype=tf.float32)

# weight & bias
W = tf.Variable(tf.random.normal([30,1]))
b = tf.Variable(tf.random.normal([1]))

# hypothesis, model, predict model, logistic regression model
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# cross entropy(loss function)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(loss)

#session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 반복학습
# 전체 데이터를 이용해서 1한 학습 => 1 epoch(에폭)
for step in range(100000):
    _, loss_val = sess.run([train, loss], feed_dict={X:train_x_data, T:train_t_data.reshape(-1,1)})
    if step % 10000 == 0:
        print('loss value : {}'.format(loss_val))

loss value : 43.123653411865234
loss value : 0.35017824172973633
loss value : 0.22318385541439056
loss value : 0.20128002762794495
loss value : 0.20841838419437408
loss value : 0.4378904700279236
loss value : 0.4759713411331177
loss value : 0.20143809914588928
loss value : 0.3645379841327667
loss value : 0.33127132058143616


In [29]:
# 정확도 측정
# validation data(test_x_data, test_t_data)를 이용해서 정확도를 측정
predict = tf.cast(H >= 0.5, dtype=tf.float32)    # true -> 1.0, False -> 0.0
correct = tf.equal(predict,T)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

accuracy_val = sess.run(accuracy, feed_dict={X:test_x_data,T:test_t_data.reshape(-1,1)})
print('Accuracy :',accuracy_val)

ERROR! Session/line number was not unique in database. History logging moved to new session 40
Accuracy : 0.94736844


In [30]:
import pandas as pd

ERROR! Session/line number was not unique in database. History logging moved to new session 41


In [54]:
# 두번째는 titanic 예제를 이용해서 logistic regression을 구현
# kaggle에서 데이터셋을 받아서 나온 결과를 kaggle에 upload해서
# 정확도를 평가받아보자!

train = pd.read_csv('C:/jupyter_home/data/train.csv')
display(train)

ERROR! Session/line number was not unique in database. History logging moved to new session 48


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [55]:
train = train.set_index('PassengerId')
display(train)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [57]:
# Cabin 결측값이 많아서 삭제
train = train.drop('Cabin', axis=1)
display(train)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S


In [58]:
# age 결측값 채우기
print(train['Age'].mean())
train['Age'].fillna(train['Age'].mean(), inplace=True)
display(train)

29.69911764705882


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,S


In [59]:
train['Embarked'].fillna(method='ffill',inplace=True)
display(train['Embarked'][826:831])

PassengerId
827    S
828    C
829    Q
830    Q
831    C
Name: Embarked, dtype: object

In [60]:
# x,t_data
train_x_data = train.drop('Survived', axis=1,inplace=False)
train_t_data = train['Survived']

In [ ]:
#모델 생성
model = linear_model.LogisticRegression()
model.fit(train_x_data,train_t_data)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LG\AppData\Local\Temp\ipykernel_3108\1676451138.py", line 3, in <module>
    model.fit(train_x_data,train_t_data)
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\sklearn\linear_model\_logistic.py", line 1514, in fit
    accept_large_sparse=solver not in ["liblinear", "sag", "saga"],
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\sklearn\base.py", line 581, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\sklearn\utils\validation.py", line 976, in check_X_y
    estimator=estimator,
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\sklearn\utils\validation.py", line 746, in check_array
    array = np.asarray(array, order=order, dtype=dtype)
  File 

In [51]:
test = pd.read_csv('C:/jupyter_home/data/test.csv')
test = test.drop('Cabin', axis=1)
test = test.set_index('PassengerId')
display(test)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S
...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S


In [52]:
test = test.dropna(subset=['Age'],axis=0,how='any')
test = test.dropna(subset=['Fare'],axis=0,how='any')

In [53]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331 entries, 892 to 1307
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    331 non-null    int64  
 1   Name      331 non-null    object 
 2   Sex       331 non-null    object 
 3   Age       331 non-null    float64
 4   SibSp     331 non-null    int64  
 5   Parch     331 non-null    int64  
 6   Ticket    331 non-null    object 
 7   Fare      331 non-null    float64
 8   Embarked  331 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 25.9+ KB


In [ ]:
predict_t_data = model.predict(test_x_data)
print(predict_t_data)   

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LG\AppData\Local\Temp\ipykernel_3108\2523868783.py", line 1, in <module>
    predict_t_data = model.predict(test_x_data)
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\sklearn\linear_model\_base.py", line 425, in predict
    scores = self.decision_function(X)
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\sklearn\linear_model\_base.py", line 405, in decision_function
    check_is_fitted(self)
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\sklearn\utils\validation.py", line 1222, in check_is_fitted
    raise NotFittedError(msg % {"name": type(estimator).__name__})
sklearn.exceptions.NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

